In [23]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
%run parse.ipynb

In [24]:
# initialize GPU to move model/tensors onto
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
print(device)

cuda:2


In [38]:
# load all the datasets
training_X, training_y, testing_X, testing_y = load_from_paths()

In [27]:
# define the MLP
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.actfn = nn.ReLU()
        self.linear2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.linear1(x)
        x = self.actfn(x)
        x = self.linear2(x)
        return x

In [28]:
# initialize the MLP
model = MLP(3, 4, 2)

# transfer model to GPU
model.to(device)

MLP(
  (linear1): Linear(in_features=3, out_features=4, bias=True)
  (actfn): ReLU()
  (linear2): Linear(in_features=4, out_features=2, bias=True)
)

In [29]:
# Define our loss function (mean squared error) to be used in the grad descent step
loss = nn.MSELoss()

# Performs the gradient descent steps
optimizer = torch.optim.SGD(model.parameters(), lr=1e-6)

In [30]:
# Template code from https://uvadlc-notebooks.readthedocs.io/en/latest/tutorial_notebooks/tutorial2/Introduction_to_PyTorch.html#The-Basics-of-PyTorch
def train_model(model, optimizer, input_notes, truth, loss_module, num_epochs=5000):
    # Set model to train mode
    model.train()

    # Training loop
    for epoch in range(num_epochs):

        ## Step 1: Move input data to device (only strictly necessary if we use GPU)
        data_inputs = data_inputs.to(device)
        data_labels = data_labels.to(device)

        ## Step 2: Run the model on the input data
        preds = model(input_notes)

        ## Step 3: Calculate the loss
        loss = loss_module(preds, truth)

        ## Step 4: Perform backpropagation
        # Before calculating the gradients, we need to ensure that they are all zero.
        # The gradients would not be overwritten, but actually added to the existing ones.
        optimizer.zero_grad()
        # Perform backpropagation
        loss.backward()

        ## Step 5: Update the parameters
        optimizer.step()

In [31]:
# train for EACH file (warning: may take a while!!)
for (input_notes, truth) in zip(training_X, training_y):
    new_input_notes = torch.Tensor(input_notes)
    new_truth = torch.Tensor(truth)

    train_model(model, optimizer, new_input_notes, new_truth, loss)

In [39]:
# this creates a list of outputs for all pieces in the testing dataset
def generate_output():
    
    testing_results = []
    
    # generating an output for each piece in the testing input dataset
    for test_input in testing_X:
        new_test_input = torch.Tensor(test_input)
        new_test_input = new_test_input.to(device)
        
        y_test = model(new_test_input)
        y_test = y_test.tolist()
        for i in range(len(y_test)):
            y_test[i].append(test_input[i][2])
        testing_results.append(y_test)

    return testing_results

In [41]:
torch.save(model.state_dict(), 'mlp.pt')